In [1]:
import pandas as pd
from cryptography.fernet import Fernet
import uuid
import os

class DataPrivacyFramework:
    def __init__(self, encryption_key):
        self.fernet = Fernet(encryption_key)
        self.anonymized_columns = set()
        self.identifier = str(uuid.uuid4())

    def anonymize_data(self, df, columns_to_anonymize):
        anonymized_df = df.copy()
        anonymized_df['_identifier'] = self.identifier
        for column in columns_to_anonymize:
            anonymized_df[column] = anonymized_df[column].apply(lambda x: '*****' if pd.notnull(x) else x)
            self.anonymized_columns.add(column)
        return anonymized_df

    def encrypt_data(self, data):
        encrypted_data = self.fernet.encrypt(data.encode())
        return encrypted_data

    def decrypt_data(self, encrypted_data):
        decrypted_data = self.fernet.decrypt(encrypted_data).decode()
        return decrypted_data

    def decrypt_column(self, encrypted_column):
        decrypted_column = encrypted_column.apply(self.decrypt_data)
        return decrypted_column

    def encrypt_confidential_data(self, df, confidential_columns):
        encrypted_df = df.copy()
        for column in confidential_columns:
            encrypted_df[column] = df[column].apply(lambda x: self.encrypt_data(str(x)) if pd.notnull(x) else x)
        return encrypted_df

    def decrypt_dataframe(self, df):
        decrypted_df = df.copy()
        for column in df.columns:
            decrypted_df[column] = self.decrypt_column(df[column])
        return decrypted_df

def generate_encryption_key():
    return Fernet.generate_key()

def load_encryption_key(file_path):
    if os.path.exists(file_path):
        with open(file_path, 'rb') as f:
            return f.read()
    else:
        return None

def save_encryption_key(encryption_key, file_path):
    with open(file_path, 'wb') as f:
        f.write(encryption_key)

In [2]:
# Example usage
if __name__ == "__main__":
    # Initialize Data Privacy Framework with encryption key
    encryption_key = Fernet.generate_key()
    privacy_framework = DataPrivacyFramework(encryption_key)

    # Load sample data
    data = {
        'ID': [101, 102, 103],
        'Name': ['John Doe', 'Jane Smith', 'Alice Johnson'],
        'Email': ['john.doe@example.com', 'jane.smith@example.com', 'alice.johnson@example.com'],
        'Phone': ['123-456-7890', '456-789-0123', '789-012-3456'],
        'Income': ['2000', '4000', '6000']
    }
    df = pd.DataFrame(data)

    # Anonymize sensitive columns (Name, Email)
    anonymized_df = privacy_framework.anonymize_data(df, ['Name', 'Email'])

    # Encrypt entire DataFrame
    encrypted_df = anonymized_df.applymap(lambda x: privacy_framework.encrypt_data(str(x)) if pd.notnull(x) else x)
    print("\nEncrypted DataFrame:")
    print(encrypted_df)

    # Decrypt entire DataFrame
    decrypted_df = encrypted_df.applymap(lambda x: privacy_framework.decrypt_data(x) if pd.notnull(x) else x)
    print("\nDecrypted DataFrame:")
    print(decrypted_df)

    # Encrypt only confidential data (Name, Email)
    encrypt_confidential = anonymized_df.copy()
    encrypt_confidential[['Name', 'Email']] = encrypt_confidential[['Name', 'Email']].applymap(lambda x: privacy_framework.encrypt_data(str(x)) if pd.notnull(x) else x)
    print("\nEncrypted Confidential Data:")
    print(encrypt_confidential)



Encrypted DataFrame:
                                                  ID  \
0  b'gAAAAABmDCYfpOJeD0wLWbGQ0VOMfRqUBjUOG9XTT4Qc...   
1  b'gAAAAABmDCYfFFHXDs0n1kgtjnLpWWndaRBbsI3656rX...   
2  b'gAAAAABmDCYfeVPwzUbVPMHdFdbWKuo5k-InpUaEd3Ij...   

                                                Name  \
0  b'gAAAAABmDCYflj_7ccqy9t2jHLgcRpmcf9AveV1ClVfB...   
1  b'gAAAAABmDCYfR4YHF0yJVUbl3WA0N31SJLau4IuzRWwI...   
2  b'gAAAAABmDCYfN7C5Qu6dOjyjRi3dpxzeW-K2URRLFYdE...   

                                               Email  \
0  b'gAAAAABmDCYfqNhTZdg6za-HsriHrmWUEk4u1MdJf2DM...   
1  b'gAAAAABmDCYf8KxlJ8M63HpZBme5uC5B8gBqcPArWnO_...   
2  b'gAAAAABmDCYfI1Nw1sod3MGzhXmfrQAVYmtQeUGcklj6...   

                                               Phone  \
0  b'gAAAAABmDCYfcBrhkBMlxFWLW9cC4S1lsxzKvSFV-LO6...   
1  b'gAAAAABmDCYfh-MQ9D3ZMTYLxj9Ycf-jOYDwlphYtK83...   
2  b'gAAAAABmDCYfnOv3PSd71mz3zZZ0kg_xdZ7ZFVt2m920...   

                                              Income  \
0  b'gAAAAABmDCYfJIZc9

In [3]:
class AccessControl:
    def __init__(self, roles=None):
        self.roles = roles if roles else {}

    def add_role(self, role_name, permissions):
        self.roles[role_name] = permissions

    def check_permission(self, role_name, permission):
        if role_name in self.roles:
            return permission in self.roles[role_name]
        else:
            return False
